<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/Candles_IB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install ib_insync
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ib_insync import IB, Stock, util
import nest_asyncio

# Apply nest_asyncio to allow multiple event loops
nest_asyncio.apply()

# Connect to Interactive Brokers
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=200)
#ib.connect('127.0.0.1', 4002, clientId=2)


# Define the stock and the historical data duration
symbol = 'AAPL'
duration = '1 Y'
resolution = '1 day'

# Fetch historical stock price data
contract = Stock(symbol, 'SMART', 'USD')
bars = ib.reqHistoricalData(contract, endDateTime='', durationStr=duration, barSizeSetting=resolution, whatToShow='TRADES', useRTH=True)
df = util.df(bars)

# Disconnect from Interactive Brokers
ib.disconnect()

# Extract closing prices
closing_prices = df['close'].values

# Perform Fourier Transform on the closing prices
frequencies = np.fft.fftshift(np.fft.fftfreq(len(closing_prices)))
spectrum = np.fft.fftshift(np.abs(np.fft.fft(closing_prices)))

# Find the dominant frequency with the highest amplitude
dominant_frequency = frequencies[np.argmax(spectrum)]

# Calculate the next candlestick prediction
prediction_period = 1  # assuming next candlestick is 1 time unit away
prediction_frequency = dominant_frequency * prediction_period
next_candlestick_prediction = np.exp(1j * 2 * np.pi * prediction_frequency)

# Determine the predicted price movement
if next_candlestick_prediction.real > 0:
    prediction = "Bullish"
elif next_candlestick_prediction.real < 0:
    prediction = "Bearish"
else:
    prediction = "Neutral"

# Plot the original closing prices, Fourier Transform spectrum, and prediction
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(df['date'], closing_prices)
ax1.set_title('Closing Prices')
ax2.plot(frequencies, spectrum)
ax2.set_title('Fourier Transform Spectrum')
ax2.axvline(x=prediction_frequency, color='red', linestyle='--', label='Dominant Frequency')
ax2.axhline(y=np.max(spectrum), color='green', linestyle='--', label='Amplitude at Dominant Frequency')
ax2.legend()
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.show()
print("Next Candlestick Prediction:", prediction)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:ib_insync.client:API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 7497)")
ERROR:ib_insync.client:Make sure API port on TWS/IBG is open


ConnectionRefusedError: ignored